In [1]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GroupShuffleSplit


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score



#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
import run_expts

#TS Fresh Parameter Settings
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.feature_extraction import MinimalFCParameters
from tsfresh.feature_extraction import EfficientFCParameters
from tsfresh.feature_extraction import extract_features

import matplotlib.patches as mpatches
from scipy import stats
import scikit_posthocs as sp

# Set the desired precision for printing
np.set_printoptions(precision=2)

#for CM plotting at the bottom
import sklearn

#for viewing pandas dataframes in full
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))


In [4]:
# What do I want to prove?  

# 1. No regional trends? Each region has 2 x 5 accuracy , 2 x 5 precision , 2 x 5 recall --> Put that into the test --> DONE 

# 2. N1 is better than other data types?  Each data type has 13 x 2 accuracy , 13 x 2 precision , 13 x 2 recall --> Put that into the test --> DONE

# 3. My modelling approach is statistically significantly better than dummy and baselines? ---> In Progress

# For each of accuracy, precision and recall we are comparing
# Can compare Ada, DT vs others
# 5(dtypes) x 13 regions for acc , prec and recall

#### Combine results dfs from baseline, dummy models and regional models

In [57]:
#Model Comparisons : 
# Dummy_uniform , Dummy_prior, Dummy_stratified
# DT_Baseline , Ada_Baseline 
# DT_Minimal , DT_Efficient , Ada_Minimal, Ada_Efficient

#sorting must be by EEG Type and Region 

In [83]:
#This cell needs to be run after the dicts have been created (below cells) 
#Ordering is a bit off ofc

print(tuned_clfs.keys())
print(baseline_dfs.keys())
print(dummy_dfs.keys())

combined_dict = {}

for df_dict in [tuned_clfs, baseline_dfs, dummy_dfs]:
    for key in df_dict.keys():
        combined_dict[key] = df_dict[key].copy()
        
print(combined_dict.keys())

dict_keys(['Minimal_Ada', 'Minimal_DT', 'Efficient_Ada', 'Efficient_DT'])
dict_keys(['baseline_Ada', 'baseline_DT'])
dict_keys(['Dummy_prior', 'Dummy_stratified', 'Dummy_uniform'])
dict_keys(['Minimal_Ada', 'Minimal_DT', 'Efficient_Ada', 'Efficient_DT', 'baseline_Ada', 'baseline_DT', 'Dummy_prior', 'Dummy_stratified', 'Dummy_uniform'])


In [79]:
tuned_clfs = {}

for TS_setting in ['Minimal' , 'Efficient'] : 
    subset_df = regional_results_tuned_df[regional_results_tuned_df['TS_Fresh_setting'] == TS_setting].copy()
    for model in ['Ada' , 'DT'] :
        tuned_clfs[ TS_setting + '_' + model] = subset_df[subset_df['Classifier'] == model].sort_values(by = ['EEG Type', 'Region']).reset_index(drop = True).copy()

In [67]:
all_results_baseline_df = joblib.load('all_baseline_results_dataframe.pkl')
cols_to_drop = ['Expt_type_num','features', 'y_true', 'y_preds']
all_results_baseline_df = all_results_baseline_df.drop(columns = cols_to_drop)

baseline_dfs = {}

for model in ['Ada' , 'DT']: 
    baseline_dfs['baseline_' + model] = all_results_baseline_df[all_results_baseline_df['Classifier'] == model].sort_values(by = ['EEG Type', 'Region']).reset_index(drop = True).copy()

In [73]:
all_results_dummy_models_df = joblib.load('all_results_dummy_models_expt_1_df.pkl')
cols_to_drop = ['Expt_type_num','features', 'y_true', 'y_preds']
all_results_dummy_models_df = all_results_dummy_models_df.drop(columns = cols_to_drop)

dummy_dfs = {}

for model in ['Dummy_prior', 'Dummy_stratified', 'Dummy_uniform']:
    dummy_dfs[model] = all_results_dummy_models_df[all_results_dummy_models_df['Classifier'] == model].sort_values(by = ['EEG Type', 'Region']).reset_index(drop = True).copy()

#### Method significance testing 

In [84]:
combined_dict
pd.concat([combined_dict[key][['Accuracy' , 'Precision' , 'Recall']].mean() for key in list(combined_dict.keys())],axis = 1 , keys =  list(combined_dict.keys()) )

,Minimal_Ada,Minimal_DT,Efficient_Ada,Efficient_DT,baseline_Ada,baseline_DT,Dummy_prior,Dummy_stratified,Dummy_uniform
Accuracy,0.515256,0.513077,0.536880,0.547436,0.466966,0.488846,0.626111,0.467222,0.400556
Precision,0.339371,0.344593,0.369349,0.418548,0.268184,0.290446,0.000000,0.302941,0.270000
Recall,0.313736,0.319487,0.368608,0.421685,0.259304,0.244945,0.000000,0.328095,0.361429


In [86]:
for metric_col_name in ['Accuracy' , 'Precision' , 'Recall']:
    data_types_data = [combined_dict[key][metric_col_name].values for key in list(combined_dict.keys())]
    
    print(metric_col_name)
    print(stats.friedmanchisquare(*data_types_data ) )
    data = np.array(data_types_data)
    
    df_nemenyi = sp.posthoc_nemenyi_friedman(data.T)
    # df_nemenyi = df_nemenyi.rename(columns = data_types , index = data_types)
    df_nemenyi.columns = list(combined_dict.keys())
    df_nemenyi.index = list(combined_dict.keys())
    display(df_nemenyi)

Accuracy
FriedmanchisquareResult(statistic=162.28378026555578, pvalue=5.324297457530246e-31)


,Minimal_Ada,Minimal_DT,Efficient_Ada,Efficient_DT,baseline_Ada,baseline_DT,Dummy_prior,Dummy_stratified,Dummy_uniform
Minimal_Ada,1.000000,0.900000,0.900000,0.633239,0.300798,0.900000,0.001000,0.232785,0.001000
Minimal_DT,0.900000,1.000000,0.900000,0.652674,0.282831,0.900000,0.001000,0.217463,0.001000
Efficient_Ada,0.900000,0.900000,1.000000,0.900000,0.009015,0.300798,0.001000,0.005602,0.001000
Efficient_DT,0.633239,0.652674,0.900000,1.000000,0.001000,0.075046,0.001912,0.001000,0.001000
baseline_Ada,0.300798,0.282831,0.009015,0.001000,1.000000,0.900000,0.001000,0.900000,0.081977
baseline_DT,0.900000,0.900000,0.300798,0.075046,0.900000,1.000000,0.001000,0.900000,0.001000
Dummy_prior,0.001000,0.001000,0.001000,0.001912,0.001000,0.001000,1.000000,0.001000,0.001000
Dummy_stratified,0.232785,0.217463,0.005602,0.001000,0.900000,0.900000,0.001000,1.000000,0.114853
Dummy_uniform,0.001000,0.001000,0.001000,0.001000,0.081977,0.001000,0.001000,0.114853,1.000000


Precision
FriedmanchisquareResult(statistic=190.4221274395762, pvalue=6.637826755377186e-37)


,Minimal_Ada,Minimal_DT,Efficient_Ada,Efficient_DT,baseline_Ada,baseline_DT,Dummy_prior,Dummy_stratified,Dummy_uniform
Minimal_Ada,1.000000,0.900000,0.900000,0.188668,0.380235,0.309932,0.001,0.900000,0.633239
Minimal_DT,0.900000,1.000000,0.900000,0.038721,0.749842,0.681824,0.001,0.900000,0.900000
Efficient_Ada,0.900000,0.900000,1.000000,0.497096,0.124703,0.092695,0.001,0.749842,0.300798
Efficient_DT,0.188668,0.038721,0.497096,1.000000,0.001000,0.001000,0.001,0.006321,0.001000
baseline_Ada,0.380235,0.749842,0.124703,0.001000,1.000000,0.900000,0.001,0.900000,0.900000
baseline_DT,0.309932,0.681824,0.092695,0.001000,0.900000,1.000000,0.001,0.900000,0.900000
Dummy_prior,0.001000,0.001000,0.001000,0.001000,0.001000,0.001000,1.000,0.001000,0.001000
Dummy_stratified,0.900000,0.900000,0.749842,0.006321,0.900000,0.900000,0.001,1.000000,0.900000
Dummy_uniform,0.633239,0.900000,0.300798,0.001000,0.900000,0.900000,0.001,0.900000,1.000000


Recall
FriedmanchisquareResult(statistic=216.57150544867503, pvalue=2.0405359772570323e-42)


,Minimal_Ada,Minimal_DT,Efficient_Ada,Efficient_DT,baseline_Ada,baseline_DT,Dummy_prior,Dummy_stratified,Dummy_uniform
Minimal_Ada,1.000000,0.900000,0.900000,0.009556,0.720693,0.163201,0.001,0.900000,0.455499
Minimal_DT,0.900000,1.000000,0.900000,0.006712,0.778993,0.202726,0.001,0.900000,0.390927
Efficient_Ada,0.900000,0.900000,1.000000,0.319146,0.075046,0.002823,0.001,0.900000,0.900000
Efficient_DT,0.009556,0.006712,0.319146,1.000000,0.001000,0.001000,0.001,0.049307,0.856726
baseline_Ada,0.720693,0.778993,0.075046,0.001000,1.000000,0.900000,0.001,0.412477,0.004121
baseline_DT,0.163201,0.202726,0.002823,0.001000,0.900000,1.000000,0.001,0.042710,0.001000
Dummy_prior,0.001000,0.001000,0.001000,0.001000,0.001000,0.001000,1.000,0.001000,0.001000
Dummy_stratified,0.900000,0.900000,0.900000,0.049307,0.412477,0.042710,0.001,1.000000,0.759557
Dummy_uniform,0.455499,0.390927,0.900000,0.856726,0.004121,0.001000,0.001,0.759557,1.000000


#### Data Type significance Testing 

In [28]:
regional_results_tuned_df = joblib.load('expt_1_results_df_tuned_regional_only.pkl')
cols_to_drop = ['Expt_type_num','features', 'y_true', 'y_preds']
regional_results_tuned_df = regional_results_tuned_df.drop(columns = cols_to_drop)

data_type_dfs = {}
data_types = ['N1','REM', 'N2', 'N3', 'Wake']

for data_type in data_types:
    #Seperate results from each data type into seperate dataframe, in equivalent order hence sorted by relevant columns
    data_type_dfs[data_type] = regional_results_tuned_df[regional_results_tuned_df['EEG Type'] == data_type].sort_values(by = ['Classifier', 'Region', 'TS_Fresh_setting']).reset_index(drop = True).copy()

#Mean accuracy, precision and recall per data_type

pd.concat([data_type_dfs[data_type][['Accuracy' , 'Precision' , 'Recall']].mean() for data_type in data_types],axis = 1 , keys = data_types )

,N1,REM,N2,N3,Wake
Accuracy,0.673077,0.515385,0.452030,0.471474,0.528846
Precision,0.583191,0.319980,0.226587,0.284271,0.425797
Recall,0.498718,0.325549,0.220513,0.292308,0.442308


In [43]:
df_nemenyi.index = data_types

In [48]:
metric_col_name = 'Accuracy'
for metric_col_name in ['Accuracy' , 'Precision' , 'Recall']:
    data_types_data = [data_type_dfs[data_type][metric_col_name].values for data_type in data_types]
    print(metric_col_name)
    print(stats.friedmanchisquare(*data_types_data ) )
    data = np.array(data_types_data)
    
    df_nemenyi = sp.posthoc_nemenyi_friedman(data.T)
    # df_nemenyi = df_nemenyi.rename(columns = data_types , index = data_types)
    df_nemenyi.columns = data_types
    df_nemenyi.index = data_types
    display(df_nemenyi)

Accuracy
FriedmanchisquareResult(statistic=88.48355899419734, pvalue=2.7642605357477034e-18)


,N1,REM,N2,N3,Wake
N1,1.000,0.001000,0.001000,0.001000,0.001000
REM,0.001,1.000000,0.451409,0.339031,0.900000
N2,0.001,0.451409,1.000000,0.900000,0.136496
N3,0.001,0.339031,0.900000,1.000000,0.087803
Wake,0.001,0.900000,0.136496,0.087803,1.000000


Precision
FriedmanchisquareResult(statistic=106.8999028182701, pvalue=3.3340993859866032e-22)


,N1,REM,N2,N3,Wake
N1,1.000,0.001000,0.001000,0.001000,0.001000
REM,0.001,1.000000,0.059057,0.900000,0.081608
N2,0.001,0.059057,1.000000,0.305285,0.001000
N3,0.001,0.900000,0.305285,1.000000,0.008969
Wake,0.001,0.081608,0.001000,0.008969,1.000000


Recall
FriedmanchisquareResult(statistic=69.27717923604315, pvalue=3.225315299344347e-14)


,N1,REM,N2,N3,Wake
N1,1.000000,0.002093,0.001000,0.001000,0.631024
REM,0.002093,1.000000,0.002971,0.560634,0.136496
N2,0.001000,0.002971,1.000000,0.216309,0.001000
N3,0.001000,0.560634,0.216309,1.000000,0.001288
Wake,0.631024,0.136496,0.001000,0.001288,1.000000


#### Regional Significance Testing

In [29]:
region_channel_dict = constants.region_to_channel_dict
regions = list(region_channel_dict.keys())

region_dfs = {}

for region in regions:
    #Seperate results from each region into seperate dataframe, in equivalent order hence sorted by relevant columns
    region_dfs[region] = regional_results_tuned_df[regional_results_tuned_df['Region'] == region].sort_values(by = ['Classifier', 'EEG Type', 'TS_Fresh_setting']).reset_index(drop = True).copy()

#Mean accuracy, precision and recall per region
pd.concat([region_dfs[region][['Accuracy' , 'Precision' , 'Recall']].mean() for region in regions],axis = 1 , keys = regions )

,Prefrontal,Frontal,Left Frontal,Right Frontal,Central,Left Central,Right Central,Left Temporal,Right Temporal,Parietal,Left Parietal,Right Parietal,Occipital
Accuracy,0.524722,0.546111,0.568611,0.517083,0.521111,0.555000,0.512361,0.486250,0.516111,0.514444,0.518056,0.527222,0.559028
Precision,0.370991,0.415725,0.418529,0.364875,0.367752,0.403028,0.344676,0.330948,0.354381,0.334699,0.319136,0.356663,0.402142
Recall,0.376071,0.397262,0.419762,0.382976,0.363929,0.381786,0.332738,0.266667,0.349405,0.336310,0.289167,0.349167,0.381190


In [49]:
for metric_col_name in ['Accuracy' , 'Precision' , 'Recall']:
    regional_data = [ region_dfs[region][metric_col_name].values for region in regions]
    print(metric_col_name)
    print(stats.friedmanchisquare(*regional_data ) )
    data = np.array(regional_data)
    
    df_nemenyi = sp.posthoc_nemenyi_friedman(data.T)
    # df_nemenyi = df_nemenyi.rename(columns = data_types , index = data_types)
    df_nemenyi.columns = regions
    df_nemenyi.index = regions
    display(df_nemenyi)

Accuracy
FriedmanchisquareResult(statistic=18.25875785066292, pvalue=0.10805665012925914)


,Prefrontal,Frontal,Left Frontal,Right Frontal,Central,Left Central,Right Central,Left Temporal,Right Temporal,Parietal,Left Parietal,Right Parietal,Occipital
Prefrontal,1.0,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000
Frontal,0.9,1.000000,0.900000,0.900000,0.900000,0.900000,0.900000,0.775855,0.900000,0.900000,0.900000,0.900000,0.900000
Left Frontal,0.9,0.900000,1.000000,0.801592,0.788723,0.900000,0.518447,0.115696,0.853073,0.685759,0.711502,0.762981,0.900000
Right Frontal,0.9,0.900000,0.801592,1.000000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000
Central,0.9,0.900000,0.788723,0.900000,1.000000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.891684
Left Central,0.9,0.900000,0.900000,0.900000,0.900000,1.000000,0.878813,0.407928,0.900000,0.900000,0.900000,0.900000,0.900000
Right Central,0.9,0.900000,0.518447,0.900000,0.900000,0.878813,1.000000,0.900000,0.900000,0.900000,0.900000,0.900000,0.621408
Left Temporal,0.9,0.775855,0.115696,0.900000,0.900000,0.407928,0.900000,1.000000,0.900000,0.900000,0.900000,0.900000,0.174686
Right Temporal,0.9,0.900000,0.853073,0.900000,0.900000,0.900000,0.900000,0.900000,1.000000,0.900000,0.900000,0.900000,0.900000
Parietal,0.9,0.900000,0.685759,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,1.000000,0.900000,0.900000,0.788723


Precision
FriedmanchisquareResult(statistic=19.137200609165124, pvalue=0.08527075995477465)


,Prefrontal,Frontal,Left Frontal,Right Frontal,Central,Left Central,Right Central,Left Temporal,Right Temporal,Parietal,Left Parietal,Right Parietal,Occipital
Prefrontal,1.0,0.900000,0.900000,0.9,0.9,0.900000,0.900000,0.900000,0.9,0.900000,0.900000,0.900000,0.900000
Frontal,0.9,1.000000,0.900000,0.9,0.9,0.900000,0.900000,0.595668,0.9,0.750112,0.531317,0.900000,0.900000
Left Frontal,0.9,0.900000,1.000000,0.9,0.9,0.900000,0.827332,0.323640,0.9,0.492512,0.263754,0.891684,0.900000
Right Frontal,0.9,0.900000,0.900000,1.0,0.9,0.900000,0.900000,0.900000,0.9,0.900000,0.900000,0.900000,0.900000
Central,0.9,0.900000,0.900000,0.9,1.0,0.900000,0.900000,0.900000,0.9,0.900000,0.900000,0.900000,0.900000
Left Central,0.9,0.900000,0.900000,0.9,0.9,1.000000,0.900000,0.724369,0.9,0.878813,0.660020,0.900000,0.900000
Right Central,0.9,0.900000,0.827332,0.9,0.9,0.900000,1.000000,0.900000,0.9,0.900000,0.900000,0.900000,0.900000
Left Temporal,0.9,0.595668,0.323640,0.9,0.9,0.724369,0.900000,1.000000,0.9,0.900000,0.900000,0.900000,0.407928
Right Temporal,0.9,0.900000,0.900000,0.9,0.9,0.900000,0.900000,0.900000,1.0,0.900000,0.900000,0.900000,0.900000
Parietal,0.9,0.750112,0.492512,0.9,0.9,0.878813,0.900000,0.900000,0.9,1.000000,0.900000,0.900000,0.569928


Recall
FriedmanchisquareResult(statistic=23.57626628075249, pvalue=0.023213464726655583)


,Prefrontal,Frontal,Left Frontal,Right Frontal,Central,Left Central,Right Central,Left Temporal,Right Temporal,Parietal,Left Parietal,Right Parietal,Occipital
Prefrontal,1.000000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.436874,0.900000,0.900000,0.465229,0.900000,0.900000
Frontal,0.900000,1.000000,0.900000,0.900000,0.900000,0.900000,0.900000,0.422442,0.900000,0.900000,0.451162,0.900000,0.900000
Left Frontal,0.900000,0.900000,1.000000,0.900000,0.900000,0.900000,0.569928,0.026176,0.685759,0.436874,0.030031,0.900000,0.900000
Right Frontal,0.900000,0.900000,0.900000,1.000000,0.900000,0.900000,0.900000,0.647151,0.900000,0.900000,0.672891,0.900000,0.900000
Central,0.900000,0.900000,0.900000,0.900000,1.000000,0.900000,0.900000,0.724369,0.900000,0.900000,0.750112,0.900000,0.900000
Left Central,0.900000,0.900000,0.900000,0.900000,0.900000,1.000000,0.900000,0.788723,0.900000,0.900000,0.814465,0.900000,0.900000
Right Central,0.900000,0.900000,0.569928,0.900000,0.900000,0.900000,1.000000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000
Left Temporal,0.436874,0.422442,0.026176,0.647151,0.724369,0.788723,0.900000,1.000000,0.900000,0.900000,0.900000,0.737242,0.531317
Right Temporal,0.900000,0.900000,0.685759,0.900000,0.900000,0.900000,0.900000,0.900000,1.000000,0.900000,0.900000,0.900000,0.900000
Parietal,0.900000,0.900000,0.436874,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,1.000000,0.900000,0.900000,0.900000


### Method significance testing Dummy vs Baseline vs Minimal vs Efficient